### Reference
- Multi-Label Classification Model From Scratch: Step-by-Step Tutorial (https://huggingface.co/blog/Valerii-Knowledgator/multi-label-classification)
- https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
- https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb

# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

In [2]:
import os
import random
import numpy as np
import torch
import transformers
from pprint import pprint
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [3]:
seed = 42
lang = 'eng'
# hf_model_id = 'google-bert/bert-base-uncased'
# hf_model_id = 'google-bert/bert-large-uncased'
hf_model_id = 'bhadresh-savani/bert-base-uncased-emotion'
# hf_model_id = 'j-hartmann/emotion-english-distilroberta-base'
# hf_model_id = 'bhadresh-savani/bert-base-go-emotion'
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'track_a_eng_70_15_15_stratify_v2'
# hf_data_config = 'track_a_eng_70_15_15_with_neutral'
project_name = f'BERT-Base-SE2025T11A-{lang}-v0.5'

In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]

splits = [*datasets.keys()]
# df = {split: pd.DataFrame(datasets[split]) for split in splits}

# cols = list(df['train'].columns)
print("Data columns:", cols)

# emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
# # neutral_emotion = df['train'][df['train'][emotion_cols].sum(axis=1) == 0]['emotion'].iloc[0]
# # emotions = emotion_cols + [neutral_emotion]
print("Emotions columns:", emotion_cols)
# print()

# print("Train DF size:", len(df['train']))
# print("Validation DF size:", len(df['val']))
# print("Testing DF size:", len(df['test']))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


(…)ack_a/eng_70_15_15_stratify_v2/train.csv:   0%|          | 0.00/198k [00:00<?, ?B/s]

(…)track_a/eng_70_15_15_stratify_v2/val.csv:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

(…)rack_a/eng_70_15_15_stratify_v2/test.csv:   0%|          | 0.00/41.2k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Data columns: ['text', 'emotion', 'anger', 'fear', 'joy', 'sad', 'surprise']
Emotions columns: ['anger', 'fear', 'joy', 'sad', 'surprise']


In [6]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:\n")
pprint(class2id, width=1)
print()
print("ID to Class:\n")
pprint(id2class, width=1)

Class to ID:

{'anger': 0,
 'fear': 1,
 'joy': 2,
 'sad': 3,
 'surprise': 4}

ID to Class:

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'sad',
 4: 'surprise'}


## Preprocess Data

In [7]:
tokenizer = AutoTokenizer.from_pretrained(hf_model_id)

In [8]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in splits}

Map:   0%|          | 0/1943 [00:00<?, ? examples/s]

Map:   0%|          | 0/412 [00:00<?, ? examples/s]

Map:   0%|          | 0/413 [00:00<?, ? examples/s]

In [9]:
# Sanity check
data = tokenized_datasets['train'][5]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: I open the door and step through while turning to close it behind me.
Emotion(s): neutral
Labels: [0.0, 0.0, 0.0, 0.0, 0.0] --> ['anger', 'fear', 'joy', 'sad', 'surprise']


In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(
    hf_model_id, 
    # num_labels=len(emotion_cols),
    # id2label=id2class, label2id=class2id,
    problem_type = "multi_label_classification",
)
print(model.classifier)

Linear(in_features=768, out_features=6, bias=True)


In [19]:
# Freeze the rest of the layers for transfer learning
# for param in model.parameters():
#     param.requires_grad = False

model.classifier = torch.nn.Linear(in_features=768, out_features=len(emotion_cols), bias=True)
print(model.classifier)

Linear(in_features=768, out_features=5, bias=True)


# Finetuning

In [14]:
def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > 0.5).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    # f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    # f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        **f1_labels_dict,
    }

In [20]:
train_args = TrainingArguments(
    # Training config
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Logging config for training
    logging_strategy='steps',
    logging_steps=100,

    # Evaluation config during training
    per_device_eval_batch_size=2,
    eval_strategy='steps',
    eval_steps=100,

    # Model saving config
    output_dir=project_name,
    save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-20-3f3975f74a33>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Label Anger,F1 Label Fear,F1 Label Joy,F1 Label Sad,F1 Label Surprise
100,0.576100,0.520723,0.537931,0.348228,0.000000,0.766798,0.455882,0.044776,0.473684
200,0.506200,0.471085,0.626160,0.440222,0.000000,0.775920,0.278261,0.649573,0.497354
300,0.460900,0.450543,0.678428,0.570455,0.229508,0.800000,0.585987,0.640625,0.596154
400,0.442100,0.463877,0.651633,0.534647,0.181818,0.779174,0.532468,0.635220,0.544554
500,0.449400,0.418639,0.653482,0.561361,0.323529,0.801556,0.596685,0.587678,0.497354
600,0.436800,0.423415,0.683240,0.564977,0.145455,0.811024,0.610169,0.675325,0.582915
700,0.401700,0.402476,0.718477,0.616068,0.210526,0.819802,0.660194,0.652174,0.737643
800,0.439400,0.411444,0.703937,0.619163,0.447368,0.804921,0.458015,0.656000,0.729508
900,0.414800,0.377619,0.724661,0.675162,0.517647,0.810916,0.659459,0.672269,0.715517
1000,0.377700,0.377536,0.736597,0.685961,0.505263,0.813559,0.662722,0.716981,0.731278


TrainOutput(global_step=2916, training_loss=0.30059622074841474, metrics={'train_runtime': 319.6749, 'train_samples_per_second': 18.234, 'train_steps_per_second': 9.122, 'total_flos': 82750655804928.0, 'train_loss': 0.30059622074841474, 'epoch': 3.0})

In [23]:
trainer.push_to_hub()

events.out.tfevents.1730117826.d8308e4c2299.920.0:   0%|          | 0.00/8.85k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1730117904.d8308e4c2299.920.1:   0%|          | 0.00/31.1k [00:00<?, ?B/s]

events.out.tfevents.1730118231.d8308e4c2299.920.2:   0%|          | 0.00/751 [00:00<?, ?B/s]

Upload 19 LFS files:   0%|          | 0/19 [00:00<?, ?it/s]

...

events.out.tfevents.1730123477.d8308e4c2299.23170.1:   0%|          | 0.00/751 [00:00<?, ?B/s]

events.out.tfevents.1730123540.d8308e4c2299.23170.2:   0%|          | 0.00/31.1k [00:00<?, ?B/s]

events.out.tfevents.1730123869.d8308e4c2299.23170.3:   0%|          | 0.00/751 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alxxtexxr/BERT-Base-SE2025T11A-eng-v0.5/commit/912566dbcb311829f2c85cb38400ea5453502729', commit_message='End of training', commit_description='', oid='912566dbcb311829f2c85cb38400ea5453502729', pr_url=None, pr_revision=None, pr_num=None)

# Evaluation

In [22]:
eval = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

import pandas as pd
pd.DataFrame(eval, index=[0])

,eval_loss,eval_f1_micro,eval_f1_macro,eval_f1_label_anger,eval_f1_label_fear,eval_f1_label_joy,eval_f1_label_sad,eval_f1_label_surprise,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.387672,0.744444,0.702767,0.556962,0.808679,0.691489,0.731707,0.725,2.9916,138.055,69.195,3.0


# Inference

In [28]:
data = datasets['val'][8]
text = data['text']
emotion_true = data['emotion']

inputs = tokenizer(text, return_tensors='pt').to(model.device)

outputs = trainer.model(**inputs)
logits = outputs.logits
probs = sigmoid(logits.squeeze().detach().cpu().numpy()) # apply sigmoid + threshold
labels_pred = (probs > 0.5).astype(int)
emotion_pred = [id2class[idx] for idx, label in enumerate(labels_pred) if label == 1.0] # turn predicted id's into actual label names

print("Text:", text)
print("True emotion(s):", emotion_true)
print("Predicted emotion(s):", ", ".join(emotion_pred))

Text: They said they don't remember any man.
True emotion(s): fear, sad, surprise
Predicted emotion(s): fear, sad, surprise
